# Sentiment & Wordcloud Analysis.
## Using Bag of Words Model.

## Data:-  Twitter Data. <br> Topic:- Corona Virus.

***
***

# Library.

In [ ]:
# Installing the library to fetch Tweets
#!pip install tweepy

In [ ]:
import tweepy
import pandas as pd

# Data Import & Preprocessing.

In [ ]:
# Variables for user credentials to access Twitter API
access_token = "1399819700457017345-K9Hhc0tfURvgRd4NNhQvej7YW3ReSs"
access_token_secret = "hUl3ANuYhrgJxYNwT2ML5weSsDC7oOEPEqYpKYOcpz0w3"
consumer_key = "qpLmZbupMFUV0qJyYdWOjoJNG"
consumer_secret = "0ltT1eyYKTxFcQJgma945JrJKJYJgQiEV8WjnQUg2GRaWY2yeO"

In [ ]:
# Connecting to twitter using credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Searching about the given keyword in all the latest tweets & pull the data
tweets_data=api.search('corona virus',count=100, lang='en', include_rts=False, tweet_mode="extended",
                      geocode="22.5726,88.3639,100km")
print ("Total number of tweets found: "+ str(len(tweets_data)))


# Creating an empty data frame to store the tweets
tweets = pd.DataFrame()

# Extracting the tweet metadata
for status in tweets_data:
    tweets=tweets.append(status._json, ignore_index=True)

In [ ]:
# Looking at the data
tweets.head(2)

In [ ]:
tweets_data= pd.DataFrame(tweets['full_text'])
tweets_data.head(3)

In [ ]:
# Looking at the tweets of few rows.
# Converting the text column as a single.
Tweets_String= str(tweets_data.loc[20:35,'full_text'].values)
Tweets_String

### There are lot junk in the Data. Like:- Twitter-Handles, Smileys, 'RT' etc..

In [ ]:
# Creating a function to clean the data

def Data_Cleaner(Input_Data):

    # Tweet Text cleaning
    import re

    # Removing the twitter usernames from tweet string
    Tweet_Texts_Cleaned=re.sub(r'@\w+', ' ', Input_Data)

    # Removing the URLS from the tweet string
    Tweet_Texts_Cleaned=re.sub(r'http\S+', ' ', Tweet_Texts_Cleaned)

    # removing all the special characters
    #Tweet_Texts_Cleaned = re.sub(r'[?|$|.|!|:|RT]',r' ',Tweet_Texts_Cleaned)

    # Removing everything which is not alphabets.
    Tweet_Texts_Cleaned= re.sub(r'[^a-z A-Z]',r'',Tweet_Texts_Cleaned)

    # Removing 'RT' from tweet string
    Tweet_Texts_Cleaned= re.sub(r'[RT]',r' ',Tweet_Texts_Cleaned)

    # Removing extra spaces in the text
    Tweet_Texts_Cleaned= re.sub(r' +', ' ', Tweet_Texts_Cleaned)

    return(Tweet_Texts_Cleaned)

In [ ]:
# Calling the Function & Cleaning the data.
tweets_data['Tweets']= tweets_data['full_text'].apply(Data_Cleaner)
tweets_data.head(3)
#str(tweets_data['Tweets'].values)

In [ ]:
tweets_data.loc[1,'Tweets']

# Sentiment Analysis & Visualization (B.O.W model).

In [ ]:
# Import sentiment words.
Positive_Words= pd.read_table('C:\\Users\\Pranab_Kumar_Paul\\Desktop\\Python_Script\\My_Script\\Python_Project(GitHub)\\UnSupervised__(Text_Data)\\Sentiment_Analysis_Data\\Positive_words.txt', encoding= 'latin-1')
Negative_Words= pd.read_table('C:\\Users\\Pranab_Kumar_Paul\\Desktop\\Python_Script\\My_Script\\Python_Project(GitHub)\\UnSupervised__(Text_Data)\\Sentiment_Analysis_Data\\Negative_words.txt', encoding= 'latin-1')

In [ ]:
## Sample positive words.
Positive_Words.head()

In [ ]:
## Sample negative words.
Negative_Words.head()

In [ ]:
# Function for Sentiment analysis using B.O.W model.
def BOW_Sentiment(inpSentence):
    
    Positive_Score= 0
    Negative_Score= 0
    Overall_Sentiment= ''
    
    # Converting every word in lowercase.
    new_sentence= inpSentence.lower()
    
    # Splitting the sentence into list of words.
    word_list= new_sentence.split()
    
    # Iterate every word and check if it is present in the positive or negative word list.
    for word in word_list:
        if (word in Negative_Words.values):
            Negative_Score+=1            
        if (word in Positive_Words.values):
            Positive_Score+=1
    
    # Checking the sentiment.
    if ((Positive_Score - Negative_Score) > 0):
        Overall_Sentiment= 'Positive'

    elif ((Positive_Score - Negative_Score) == 0):
        Overall_Sentiment= 'Neutral'

    else:
        Overall_Sentiment= 'Negative'

    return(Overall_Sentiment)

In [ ]:
## Applying the function.
tweets_data['Sentiment']=tweets_data['Tweets'].apply(BOW_Sentiment)

In [ ]:
## Sample sentiment.
tweets_data.head(3)

In [ ]:
# Visualizing the overall sentiment distribution

import matplotlib.pyplot as plt
fig, subPlot =plt.subplots(nrows=1, ncols=2, figsize=(10,4),dpi= 90)
fig.suptitle("Sentiment analysis of The Tweets #Corona_Virus",fontsize= 20)

# Grouping the data
#GroupedData=tweets_data['Sentiment'].value_counts()
GroupedData=tweets_data.groupby('Sentiment').size()

# Creating the charts
GroupedData.plot(kind='bar', ax=subPlot[0], color=['crimson','lightblue', 'green'],rot= 0)
GroupedData.plot(kind='pie', ax=subPlot[1], colors=['crimson','lightblue', 'green'], autopct= '%.2f %%',
                         startangle = 0, explode = (0.1,0.0,0.0))    
plt.show()

# Wordcloud Analysis of the Negative Tweets

In [ ]:
#tweets_data[['Tweets','Sentiment']][tweets_data['Sentiment']=='Negative']

Negative_Sentiment= str(tweets_data['Tweets'][tweets_data['Sentiment']=='Negative'].values)
Negative_Sentiment

In [ ]:
# Finding the important word combinations using textblob
from textblob import TextBlob

# Converting the sample text to a blob
SampleTextInBlobFormat = TextBlob(Negative_Sentiment)

# Finding the noun phrases (important keywords combination) in the text, to understand what entities are being talked about
NounPhrases=SampleTextInBlobFormat.noun_phrases

# Creating an empty list to hold new values
# combining the noun phrases using underscore to visualize it as wordcloud
NewNounList=[]
for words in NounPhrases:
    NewNounList.append(words.replace(" ", "_"))
    
# Splitting the text to make list of words
#WordList=str(NewNounList).split()    

In [ ]:
# Looking at the Few noun_phrases
NewNounList[1:6]

In [ ]:
# Finding out the frequency of Top 10 words in text

# Creating a Dataframe
WordData=pd.DataFrame(data= pd.Series(NewNounList), columns=['words'])

# Finding the top frequency words
print(WordData.groupby('words').size().nlargest(15))


In [ ]:
# Converting list into a string to plot wordcloud
NewNounString=' '.join(NewNounList)
#print('##### Important word combinations ####')
#print(NewNounString)

########################################################################
# Plotting the wordcloud
%matplotlib inline
import matplotlib.pyplot as plt

from wordcloud import WordCloud, STOPWORDS

# Creating a custom list of stopwords
customStopwords=list(STOPWORDS) + ['corona_virus','corona',"cor'",'corona_virus_provides']

wordcloudimage = WordCloud( max_words=100,
                            font_step=2 ,
                            max_font_size=500,
                            stopwords=customStopwords,
                            background_color='black',
                            width=2500,
                            height=1000
                          ).generate(NewNounString)

plt.figure(figsize=(15,7))
plt.imshow(wordcloudimage)
plt.axis("off")
plt.show()